# F&F Lab 2 — HyperOpt

In [12]:
import csv
import numpy

## What is this?

_HyperOpt_ is an abbreviation for _Hyperparameter optimisation_, which is exactly what it sounds like: it's the process of automatically tuning hyperparameters to get the best performance from your model.

Defining the difference between hyperparameters and normal parameters can be somewhat tricky (in the fully general sense; it's usually straight forward for any given model). In fact, if you Google for the difference many people define hyperparameters as the part of the algorithm you set manually and the parameters as the part set by the data; this clearly contradicts the existence of _HyperOpt_, and is a good lesson in why you should avoid websites like _towardsdatascience.com_ (it's more wrong than right). However, this does capture it in some sense: parameters are set by the data and hyperparameters are set by something else. In the case of _HyperOpt_ you use a _validation_ set for learning the hyperparameters; it needs to be a separate data set because hyperparameters can almost always be set such that overfitting occurs. You will then need a third _test_ set to determine the final, true, performance of the complete model because, in tuning the model to fit the _validation_ set, you risk overfitting to the _validation_ set via the hyperparameters! The rule is that, as soon as a data set influences the model then measuring performance with it is no longer valid, though may still be useful. You may extrapolate that to what it means for a researcher to work on the same data set for any length of time, and hence realise that in practise following this rule exactly is unreasonable.

The HyperOpt is going to be done using a genetic algorithm; just follow the slides and do whatever (it will probably work!). Part of the point here is to get a feel for the options, so try out whatever seems like a reasonable idea and compare your approach to your neighbours: see whose approach can reliably find a good solution in the least time. In addition, you're going to implement a ML algorithm which you probably haven't seen before "_Random Kitchen Sinks_". This is because it's an example of the idea that, under the right circumstances, you can actually replace optimisation with randomisation, which is useful in some situations and hence worth introducing.

## Load data

In [13]:
for name in ['train', 'validation', 'test']:
    x = []
    y = []

    with open(name + '.csv', newline='') as fin:
        reader = csv.reader(fin)
        first = True
        for row in reader:
            if first:
                first = False
                continue

            x.append([float(v) for v in row[:-1]])
            y.append(float(row[-1]))

    x = numpy.array(x)
    y = numpy.array(y)
    
    # If you haven't seen this before this is a techneque to write variables by name,
    # where that name is a string constructed at runtime — it's a bad idea almost always
    # and only hapenning here because Jupyter is also a bad idea...
    globals()[name + '_x'] = x
    globals()[name + '_y'] = y

    
print(f'features = {train_x.shape[1]}')
print(f'train examples = {train_x.shape[0]}')
print(f'validation examples = {validation_x.shape[0]}')
print(f'test examples = {test_x.shape[0]}')

features = 11
train examples = 8192
validation examples = 8192
test examples = 8192


## Baseline

You always want a baseline performance measure, so you know what bad looks like! In this case, what happens if you always predict that y is the mean of the training data.

In [14]:
mean_y = train_y.mean()

mean_rmse = numpy.sqrt(numpy.square(test_y - mean_y).mean())
print(f'Baseline test root mean square error = {mean_rmse}')

Baseline test root mean square error = 3.3468054970123737


## 1. Random Kitchen Sinks

Consider a neural network with a single hidden layer:
$$\text{input} \rightarrow \text{hidden} \rightarrow \text{output}$$

There are two blocks of weights, specifically the first block connecting the input to the hidden layer and then the second block connecting the hidden layer to the output layer; traditionally all weights would be fitted to the training set using back propagation combined with stochastic gradient descent and an optimiser such as adam.

It happens to be the case that instead of doing this you may just randomise the first block of weights instead; to understand why you need to dive into random basis functions, which is probably a bit much for now, but if you really want to then you can see the seqeuence of papers that built up to this idea on one of the [authors websites](https://people.eecs.berkeley.edu/~brecht/kitchensinks.html). The useful consequences of doing this are

* The optimisation of just the last layer is linear regression: the problem is now convex and much easier to solve. No backpropagation!
* The choice of non-linearity is no longer limited by the requirement that it can be optimised with gradient descent; periodic non-linearities (e.g. sine) are often choosen because they keep changing through the space and hence work better. When sine is choosen it's an example of something called _random Fourier features_.
* You don't have to store the weights of the first layer; instead you can store the seed that a psuedo random number generator (pRNG) uses to generate them and do so every time you need them. There is no reason to do that here as storage isn't tight on a modern computer; more a concern for low power devices or saving disk space.
* They are a universal function approximator, i.e. can represent any function as the width of the hidden layer goes to infinity.

there are also some disadvantages

* Many non-linearity won't work on the output layer (has to be invertable).
* You're limited to only one hidden layer, which typically has to be very wide — multiple layers has a combinatoric effect, that mean you get away with less units and that's not something that can be exploited here.
* They like to overfit, but the HyperOpt should minimise that. In fact, this weakness makes them an ideal candidate for trying out HyperOpt!

Your first task is to implement this algortihm. The trainning subsection below describes it step-by-step; don't forget you'll need a predict method as well.


### Training

1. Randomly generate a weight matrix for the first layer. There is techneque to this: you want to pick the weights going into each unit to be vectors with the direction evenly distributed but the length uniformly distributed in a range that you specify with a pair of hyperparameters (low and high). This is done in three steps
    1. Fill the weight matrix with draws from a standard Gaussian.
    2. Normalise all weights for a single hidden unit so it's weight vector is of length 1 — this results in unit length vectors evenly distributed on the surface of the unit hyper-sphere.
    3. Scale each hidden units weight vector by a draw from a uniform distribution, covering the required range.
2. Generate a "new" set of input features, by running the original input features through the weight matrix followed by applying the non-linearity, which will be `numpy.sin()`. Try and avoid looping here, as that will be very slow.
3. Concatenate a one to the end of the feature vector — this will act as the bias term in the following step.
4. You now have a linear regression problem; solve it to generate the weights that connect the hidden layer to the output (it will be a vector because there is only one output).


### Notes

* A framework class has been provided; you may choose to use it or bin it and replace it with your own. But it might help to already have a structure to work with.
* No need to use `PyTorch` or equivalent — `numpy` is enough!
* Make sure the code broadcasts as much as is reasonable: model answer runs in under a second on one particular computer.
* To generate random numbers you will want to create a random number generator with something like `rng = numpy.random.default_rng(0)` (0 fixes the seed, so it produces the same sequence of "random" numbers every time) and then call the many methods it provides for generation, e.g. `rng.standard_normal()` if you want a draw from the standard Gaussian (it takes a `size` parameter if you want many such values).
* Double check you've normalised the correct dimension when generating the weights: get it wrong and it will destroy performance. Most of the other mistakes will make it crash instead:-)
* `numpy.linalg.lstsq()` solves linear equations!
* While `numpy.einsum()` may cause some existential terror, it's very useful and worth learning. For instance, `numpy.einsum('rc,...c->...r', a, x)` does a matrix multiplication on every vector in x, even if x contains many vectors; this is what each layer of a NN is doing. Yes, the same thing can be done with `numpy.tensordot()`, but this is arguably clearer and more general.

In [15]:
class RandomKitchenSink:
    """Regresses a single number given an input feature vector."""
    
    def __init__(self, low, high, input_width = 11, hidden_width = 512):
        """Initialises the model with the random vector length (low to high) of the
        random weights connecting the input to the hidden layer, plus how many input
        units there are and how many hidden units."""
        self._low = low
        self._high = high
        self._input_width = input_width
        self._hidden_width = hidden_width
    
    
    def fit(self, x, y):
        """Fits the model given x (a data matrix, indexed [exemplar, feature])
        and y (a vector, indexed [exemplar])."""
        
        # Generate random weight matrix between input and hidden layer

        # Send the x values through the first half of the NN
        
        
    
    
    def predict(self, x):
        """Given x predicts y. Only needs to accept data matrices indexed [exemplar, feature]."""
        
        # Code me!


## Testing

Obviously, you want to test an algorithm before taking it further — the below runs on the training set and calculates the RMSE on the test set with `low` set to `0` and `high` set to `1`, with `512` hidden units. If you've changed the framework you may need to adapt the code.

In [16]:
model = RandomKitchenSink(0, 1)
model.fit(train_x, train_y)

train_pred_y = model.predict(train_x)
test_rmse = numpy.sqrt(numpy.square(train_pred_y - train_y).mean())
print(f'Train root mean square error = {test_rmse}')

test_pred_y = model.predict(test_x)
test_rmse = numpy.sqrt(numpy.square(test_pred_y - test_y).mean())
print(f'Test root mean square error = {test_rmse}')

TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

## 2. HyperOpt

Now optimise the two hyperparameters, low and high! That really is the extent of your instructions; there is an assumption you'll try a genetic algorithm but feel free to try anything suitable from the optimisation lecture. If you find yourself in a group aim to all try different variations. Keep in mind that `low` and `high` have an ordering constraint and should be positive; you may want to consider an alternate representation so you don't have to enforce these constraints.

Steps:
1. Write a function that takes low and high as parameters and returns a measure of how well they work, by training the algoroithm on the training set and then using the validation set to measure performance for the HyperOpt.
2. Write a function that generates random pairs of low/high, or whatever representation is used; this is used to initialise the population of solutions and implicitely defines the search space. If you're unsure of what a sensible search space is then try some numbers out in the above testing code to get a feel for what's plausible.
3. Write an optimisation loop that does the steps of a genetic algorithm: hill climbing, mutation and selection.
4. Include reporting (aka `print()`) so you can see what it's doing as it runs.
5. After the loop has ended report the train, validation and test performance: if it has worked then these should be extremely close to each other, as the point is to avoid the gap that indicates over/under fitting.

In [ ]:

# Code me!
